In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
print(tf.__version__)

## 1. Загрузка данных

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#загружаем данные для обучения
train_df = pd.read_csv("/content/train.csv", sep = ';',on_bad_lines='skip',encoding="cp1251")
train_df

In [ ]:
# Посмотрим распределение по жанрам

age_count_df = train_df[['Название книги','Возраст']].groupby('Возраст').agg('count').sort_values('Название книги', ascending = False)

age_count_df.reset_index(inplace = True)
age_count_df.rename(columns ={'Название книги':'Age_Cnt'}, inplace = True)

age_count_df.plot.bar(y = 'Age_Cnt', x = 'Возраст')

age_count_df

In [ ]:
train_df = train_df.merge(age_count_df, how = 'left', left_on='Возраст', right_on='Возраст')

train_df

In [ ]:
# Уберем редко встречающиеся варианты

Age_Cnt_MIN = 50

train_df = train_df [train_df['Age_Cnt'] > Age_Cnt_MIN]

train_df

In [ ]:

test_df = pd.read_csv('/content/test.csv', sep = ';',on_bad_lines='skip',encoding="cp1251")

test_df

In [ ]:
test_df = test_df.merge(age_count_df, how = 'left', left_on='Возраст', right_on='Возраст')

test_df

In [ ]:
Age_Cnt_MIN = 50

test_df = test_df [test_df['Age_Cnt'] > Age_Cnt_MIN]

test_df

## 2. Подготовка данных для модели


In [ ]:
def tokenize_text (p_raw_text, p_stop_words):
    '''Функция для токенизации текста

    :param p_raw_text: исходная текстовая строка
    :param p_stop_words: список стоп слов
    '''
    
    tokenized_str = nltk.word_tokenize(p_raw_text)
    tokens = [i.lower() for i in tokenized_str if ( i not in string.punctuation )]
    filtered_tokens = [i for i in tokens if ( i not in p_stop_words )]
    
    return filtered_tokens

#tokenize_text

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
#разбиваем описание на токены
import nltk
import re
nltk.download('punkt')
from nltk.corpus import stopwords

train_df['Synopsis_tokenized'] = train_df['Synopsis'].apply(lambda x:tokenize_text(x, stopwords.words('russian')))

test_df['Synopsis_tokenized'] = test_df['Synopsis'].apply(lambda x:tokenize_text(x, stopwords.words('russian')))

In [ ]:
# создаем словарь

#словарь, составленный из описаний фильмов <word>:<id>
vocabulary = {}

max_val = 1000000

#добавляем зарезервированные слова
vocabulary["<PAD>"] = max_val + 2
vocabulary["<START>"] = max_val + 1
vocabulary["<UNKNOWN>"] = max_val

#посчитаем слова
for tokens in train_df.Synopsis_tokenized:
    for word in tokens:
        if word not in vocabulary.keys():
            vocabulary[word] = 1
        else:
            vocabulary[word] = vocabulary[word] + 1
            
#отсортируем слова по частоте
vocabulary = {k: v for k, v in sorted(vocabulary.items(), key=lambda item: item[1], reverse = True)}

#проиндексируем слова
cnt = 0
for k in vocabulary.keys():
    vocabulary[k] = cnt
    cnt = cnt + 1
#for

vocabulary.pop('книге')
vocabulary.pop('хх')
vocabulary.pop('“')
vocabulary.pop('”')
print('Количестов слов в словаре:',len(vocabulary))
print()
vocabulary

In [ ]:
#создаем облегченный словарь для обучения
vocab_light = {}
for k, v in vocabulary.items():
    if v < 10000:
        vocab_light[k] = v

vocab_light

In [ ]:
#описываем функции кодирования/декодирования слов

def encode_tokens (p_tokens, p_vocabulary):
    '''Кодирования токенов (слов) в индексы словаря
    
    :param p_tokens: список токенов
    :param p_vocabulary: словарь <word>:<id>, c обязательными значениями {<PAD>:0, <START>:1, <UNKNOWN>: 2}
    '''
    res = []
     
    res = [p_vocabulary.get(word, p_vocabulary['<UNKNOWN>']) for word in p_tokens]
    
    return [p_vocabulary['<START>']] + res

#encode_tokens

def dencode_tokens (p_encoded_tokens, p_vocabulary):
    '''Декодирование токенов: индексы словаря -> в тоекны (слова)
    
    :p_encoded_tokens: список индексов словаря
    :param p_vocabulary: словарь <word>:<id>, c обязательными значениями {<PAD>:0, <START>:1, <UNKNOWN>: 2}
    '''
    
    res = []
    
    for index in p_encoded_tokens: 
        for word, v_index in p_vocabulary.items():
            if index == v_index:
                res.append(word)
                break
            #if
    
    return res
    


In [ ]:
#кодируем описание фильмов

train_df['Synopsis_encoded'] = train_df['Synopsis_tokenized'].apply(lambda x: encode_tokens (x, vocab_light))

train_df

In [ ]:
test_df['Synopsis_encoded'] = test_df['Synopsis_tokenized'].apply(lambda x: encode_tokens (x, vocab_light))

test_df

In [ ]:
#Готовим данные для обучения
train_data = train_df.Synopsis_encoded.to_numpy()
train_labels = pd.get_dummies(train_df['Возраст']).values

test_data = test_df.Synopsis_encoded.to_numpy()
test_labels = pd.get_dummies(test_df['Возраст']).values

print(len(train_data))
print(len(train_labels))
print(len(test_data))
print(len(test_labels))

In [ ]:
# Посчитаем среднюю длинну описания, чтобы определить длинну последовательности
train_df['Synopsis_len'] = train_df['Synopsis_encoded'].apply (len)

print ('минимальная длина описания:', train_df.Synopsis_len.min())
print ('средняя длина описания:', round(train_df.Synopsis_len.mean()))
print ('максимальная длина описания:', train_df.Synopsis_len.max())

plt.hist(train_df.Synopsis_len, density = True)

In [ ]:
# Приведем все цепочки в датасете к одной длине с помощью паддинга

MAX_SEQ_LEN = 50

train_data = tf.keras.preprocessing.sequence.pad_sequences(
    train_data,
    value= vocabulary['<PAD>'],
    padding= 'post',
    maxlen= MAX_SEQ_LEN)

test_data = tf.keras.preprocessing.sequence.pad_sequences(
    test_data,
    value= vocabulary['<PAD>'],
    padding= 'post',
    maxlen= MAX_SEQ_LEN)

print('Тернировочные данные:')
print(train_data.shape)
print(train_data[0])
print()
print('Тестовые данные:')
print(test_data.shape)
print(test_data[0])

In [ ]:
#Разбьем обучающий датасет на обучающий и валидационный

partial_x_train, x_val, partial_y_train, y_val = train_test_split(train_data, train_labels, 
                                                                  test_size = 0.05, random_state = 42)

print(partial_x_train.shape, partial_y_train.shape)
print(x_val.shape, y_val.shape)

## 3. Создание и обучение модели

In [ ]:
from  tensorflow.keras.utils import get_custom_objects
from keras.layers import Activation

In [ ]:
# Создадим рекурентную модель для классификации

VOCAB_SIZE = len(vocab_light)
EMB_SIZE = 32
CLASS_NUM = y_val.shape[1]

print(VOCAB_SIZE)
print(EMB_SIZE)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE),
    tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(EMB_SIZE, return_sequences=True, dropout=0.5, recurrent_dropout=0.5)),
    tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(EMB_SIZE, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)),
    tf.keras.layers.Dense(CLASS_NUM, activation= 'sigmoid'),
])


In [ ]:
#Обучение модели
BATCH_SIZE = 8 #8 77.59% / 
NUM_EPOCHS = 5

#Настраиваем объект для сохранения результатов работы модели
cpt_path = '/content/14_text_classifier.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(cpt_path, monitor='acc', verbose=1, save_best_only= True, mode='max')

model.compile(loss= 'binomial_crossentropy', optimizer='adam', metrics=['acc'])

from tensorflow.keras.utils import to_categorical

history= model.fit(partial_x_train, partial_y_train, validation_data= (x_val, y_val), 
                   epochs= NUM_EPOCHS, batch_size= BATCH_SIZE, verbose= 1,
                   callbacks=[checkpoint])


In [ ]:
# оценим качество на тестовом датасете

results = model.evaluate(test_data, test_labels)

print('Test loss: {:.4f}'.format(results[0]))
print('Test accuracy: {:.2f} %'.format(results[1]*100))